In [9]:
import pickle
import gzip
import random
import numpy as np

In [10]:
def load_data():
    f = gzip.open('/Users/Aarushi/Desktop/neural-networks-and-deep-learning/data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [11]:
def load_data():
    f = gzip.open('/Users/Aarushi/Desktop/neural-networks-and-deep-learning/data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [12]:
class Network(object):

    def __init__(self, sizes):
       
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
     
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
      
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):        
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):     
        return (output_activations-y)

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


In [13]:
training_data, validation_data, test_data = load_data_wrapper()

In [14]:
net = Network([784, 30, 10])

In [15]:
net.SGD(training_data, 30, 10, 3.0, test_data = test_data)

Epoch 0: 9014 / 10000
Epoch 1: 9199 / 10000
Epoch 2: 9302 / 10000
Epoch 3: 9330 / 10000
Epoch 4: 9347 / 10000
Epoch 5: 9392 / 10000
Epoch 6: 9394 / 10000
Epoch 7: 9432 / 10000
Epoch 8: 9413 / 10000
Epoch 9: 9435 / 10000
Epoch 10: 9471 / 10000
Epoch 11: 9462 / 10000
Epoch 12: 9491 / 10000
Epoch 13: 9493 / 10000
Epoch 14: 9504 / 10000
Epoch 15: 9491 / 10000
Epoch 16: 9507 / 10000
Epoch 17: 9509 / 10000
Epoch 18: 9508 / 10000
Epoch 19: 9513 / 10000
Epoch 20: 9496 / 10000
Epoch 21: 9500 / 10000
Epoch 22: 9509 / 10000
Epoch 23: 9490 / 10000
Epoch 24: 9527 / 10000
Epoch 25: 9498 / 10000
Epoch 26: 9516 / 10000
Epoch 27: 9520 / 10000
Epoch 28: 9518 / 10000
Epoch 29: 9487 / 10000


In [16]:
net = Network([784, 100, 10])

In [17]:
net.SGD(training_data, 30, 10, 100.0, test_data = test_data)

Epoch 0: 892 / 10000
Epoch 1: 892 / 10000
Epoch 2: 892 / 10000
Epoch 3: 892 / 10000
Epoch 4: 892 / 10000
Epoch 5: 892 / 10000
Epoch 6: 892 / 10000
Epoch 7: 892 / 10000
Epoch 8: 892 / 10000
Epoch 9: 892 / 10000
Epoch 10: 892 / 10000
Epoch 11: 892 / 10000
Epoch 12: 892 / 10000
Epoch 13: 892 / 10000
Epoch 14: 892 / 10000
Epoch 15: 892 / 10000
Epoch 16: 892 / 10000
Epoch 17: 892 / 10000
Epoch 18: 892 / 10000
Epoch 19: 892 / 10000
Epoch 20: 892 / 10000
Epoch 21: 892 / 10000
Epoch 22: 892 / 10000
Epoch 23: 892 / 10000
Epoch 24: 892 / 10000
Epoch 25: 892 / 10000
Epoch 26: 892 / 10000
Epoch 27: 892 / 10000
Epoch 28: 892 / 10000
Epoch 29: 892 / 10000


In [18]:
net = Network([784, 0 , 10])

In [19]:
net.SGD(training_data, 30, 10, 100.0, test_data = test_data)

Epoch 0: 1010 / 10000
Epoch 1: 1010 / 10000
Epoch 2: 1009 / 10000
Epoch 3: 1009 / 10000
Epoch 4: 974 / 10000
Epoch 5: 1009 / 10000
Epoch 6: 1009 / 10000
Epoch 7: 1135 / 10000
Epoch 8: 1135 / 10000
Epoch 9: 974 / 10000
Epoch 10: 1135 / 10000
Epoch 11: 974 / 10000
Epoch 12: 1028 / 10000
Epoch 13: 1028 / 10000
Epoch 14: 1028 / 10000
Epoch 15: 1028 / 10000
Epoch 16: 1010 / 10000
Epoch 17: 1010 / 10000
Epoch 18: 980 / 10000
Epoch 19: 1135 / 10000
Epoch 20: 1010 / 10000
Epoch 21: 1010 / 10000
Epoch 22: 982 / 10000
Epoch 23: 982 / 10000
Epoch 24: 892 / 10000
Epoch 25: 892 / 10000
Epoch 26: 892 / 10000
Epoch 27: 982 / 10000
Epoch 28: 1010 / 10000
Epoch 29: 974 / 10000
